# Generate fort.67 Notebook
<img style="float:right;" src="https://irr2.gmu.edu/inc/AIETemplate/MasonM.png" width=150px>

### Description:

 - Read multiple shapefiles and compile them to a single shapefile
 
### Directions:
1. Import Libraries and Assign Variables
2. Define root directory
3. Run


In [1]:
import pandas as pd
import numpy as np
import os
from decimal import Decimal

In [2]:
root_dir = r'C:\Users\tmiesse\Work\climate_grib'
file = 'fort.67.asc'

In [3]:
f67 = os.path.join(root_dir,file)
f14 = os.path.join(root_dir,'fort.14')
fnew= os.path.join(root_dir,'fort.67.new')

In [4]:
def find_columns(data):
    data2 = []
    for f in data.split(' '):
        if f != '':
            data2.append(f)   
    return data2

In [5]:
node_id,data,label=[],[],[]
with open(f67,'r') as fin:
    lines = fin.readlines()
for i in range(9,len(lines)):
    data.append(find_columns(lines[i])[2].split('\n')[0])
    node_id.append(find_columns(lines[i])[1])
    label.append(find_columns(lines[i])[0])

In [6]:
lines

[' Major:   1 Minor:   2 Rev:   0\n',
 'imhs =   111112\n',
 'time =    0.9374400000000000E+07\n',
 'iths =    9374400\n',
 'NP_G =     381164\n',
 'NE_G =     750479\n',
 'NP_A =     381164\n',
 'NE_A =     750479\n',
 '#---- ETA1 ----\n',
 '      ETA1       1:    3.9557633972E+01\n',
 '      ETA1       2:    4.0512471771E+01\n',
 '      ETA1       3:    5.4754064179E+01\n',
 '      ETA1       4:    2.5022111511E+01\n',
 '      ETA1       5:    2.2844218826E+01\n',
 '      ETA1       6:    2.3800971603E+01\n',
 '      ETA1       7:    3.2347753143E+01\n',
 '      ETA1       8:    4.0191251373E+01\n',
 '      ETA1       9:    4.1093003845E+01\n',
 '      ETA1      10:    6.0737901306E+01\n',
 '      ETA1      11:    2.8778207398E+01\n',
 '      ETA1      12:    2.0483319855E+01\n',
 '      ETA1      13:    1.8696160889E+01\n',
 '      ETA1      14:    2.8069566345E+01\n',
 '      ETA1      15:    2.8902471161E+01\n',
 '      ETA1      16:    3.3120847321E+01\n',
 '      ETA1      17:  

In [7]:
table67 = pd.DataFrame()
table67.insert(0,'label',label)
table67.insert(1,'node',node_id)
table67.insert(2,'data',data)
table67.head(2)

,label,node,data
0,ETA1,1:,3.9557633972E+01
1,ETA1,2:,4.0512471771E+01


In [8]:
nodesx, nodesy, value, node_id, node_name, loc = [], [], [], [], [], []
with open(f14, 'r') as f:
    lines = f.readlines()
    for i in range(0,len(lines)):
        line = lines[i]
        nodes = int(lines[1].split(' ')[2])
        if i>1 and i<nodes+2:
            loc.append(line.strip().split('\n')[0])
for i in range(len(loc)):
    node_id.append(find_columns(loc[i])[0])
    nodesx.append(find_columns(loc[i])[1])
    nodesy.append(find_columns(loc[i])[2])
    value.append(find_columns(loc[i])[3])
node_name = {'node_id':node_id}
table = pd.DataFrame(node_name)
table.insert(1,'node_x',nodesx)
table.insert(2,'node_y',nodesy)
table.insert(3,'value',value)
table.head(2)

,node_id,node_x,node_y,value
0,1,-76.9454281377,39.0026000108,-39.4576339722
1,2,-76.9465507379,38.9968142640,-40.4124717712


In [9]:
label2 = list(table67['label'].unique())
df = table67[table67['label'] == label2[0]]

In [10]:
df2 = pd.DataFrame(table['node_id'])
df2.insert(1,'bias',10)
df2.head(2)

,node_id,bias
0,1,10
1,2,10


In [11]:
df3 = df

In [12]:
df.head()

,label,node,data
0,ETA1,1:,3.9557633972E+01
1,ETA1,2:,4.0512471771E+01
2,ETA1,3:,5.4754064179E+01
3,ETA1,4:,2.5022111511E+01
4,ETA1,5:,2.2844218826E+01


In [13]:
df = df.replace(df.loc[table['value'].astype('float')<0.0]['data'],
                df[table['value'].astype('float')<0.0]['data'].astype('float') + df2[table['value'].astype('float')<0.0]['bias'])

In [14]:
df.head()

,label,node,data
0,ETA1,1:,3.9557633972E+01
1,ETA1,2:,4.0512471771E+01
2,ETA1,3:,5.4754064179E+01
3,ETA1,4:,2.5022111511E+01
4,ETA1,5:,2.2844218826E+01


In [ ]:
for l in label2[0:2,2]:
    df = table67[table67['label'] == label2[l]]
    df = df.replace(df.loc[table['value'].astype('float')<0.0]['data'],
                    df[table['value'].astype('float')<0.0]['data'].astype('float') + df2[table['value'].astype('float')<0.0]['bias'])
    table67 = table67.replace(table67.loc[df['label']])
    
    
    

In [15]:
df.head()

,label,node,data
0,ETA1,1:,3.9557633972E+01
1,ETA1,2:,4.0512471771E+01
2,ETA1,3:,5.4754064179E+01
3,ETA1,4:,2.5022111511E+01
4,ETA1,5:,2.2844218826E+01


In [16]:
for l in label2[0:1]:
    table67.loc[table67['label'] == l]['data'] = df['data']

C:\Users\tmiesse\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [145]:
table67.head()

,label,node,data
0,ETA1,1:,3.9557633972E+01
1,ETA1,2:,4.0512471771E+01
2,ETA1,3:,5.4754064179E+01
3,ETA1,4:,2.5022111511E+01
4,ETA1,5:,2.2844218826E+01


In [149]:
new = table67.replace(table67.loc[table67['label'] == l]['data'],df['data'])

In [150]:
new.head()

,label,node,data
0,ETA1,1:,3.9557633972E+01
1,ETA1,2:,4.0512471771E+01
2,ETA1,3:,5.4754064179E+01
3,ETA1,4:,2.5022111511E+01
4,ETA1,5:,2.2844218826E+01


In [82]:
ele = float(data)/100
ele
'{:.10E}'.format(new)

TypeError: float() argument must be a string or a number, not 'list'

In [ ]:
with open(fnew,'w') as fout:
    for i in range(0,len(lines)):
        fout.write(lines[i]+'new')